# Выбор локации для скважины "ГлавРосГосНефть"
## Постановка задачи

Заказчик проекта - «ГлавРосГосНефть». Поставленная задача - нужно решить, где бурить новую скважину.

Обычно выбор локации происходит так:
* В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
* Строят модель для предсказания объёма запасов в новых скважинах;
* Выбирают скважины с самыми высокими оценками значений;
* Определяют регион с максимальной суммарной прибылью отобранных скважин.

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

В качестве исходных данных предоставлены пробы нефти в трёх регионах; характеристики для каждой скважины в регионе уже известны. Исходные данные предоставлены в виде трех датасетов одинаковой структуры:

* `id` — уникальный идентификатор скважины;
* `f0`, `f1`, `f2` — значимые признаки;
* `product` — объём запасов в скважине (тыс. баррелей).

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

Нужно построить модель линейной регрессии для определения региона, где добыча принесёт наибольшую прибыль; после этого возможную прибыль и риски техникой Bootstrap. После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%, а затем выбрать среди них регион с наибольшей средней прибылью.

Дополнительные условия:
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* Текущая цена на баррель сырья - 450 рублей; доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

# Импорт библиотек и данных
Импортируем исходные датасеты в переменные df_0, df_1 и df_2 и ознакомимся с данными.

Исходные данные представляют собой три идеальных датасета на 100_000 строк и 5 столбцов каждый. Структура всех трех датасетов одинакова: есть столбец с `id` и четыре столбца с численными признаками в формате `float`. Названия и типы столбцов одинаковы. Типы данных адекватны, пропусков нет.

Столбцы с `id` неинформативны, их нужно будет удалить. 

Данные адекватны и пригодны для анализа.

In [155]:
from pathlib import Path

import pandas as pd
pd.set_option('display.float_format', '{:,.4f}'.format)

#import matplotlib.pyplot as plt

#import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from scipy import stats as st


In [156]:
my_path = Path('/home/klarazetkin/Documents/yandex/module_2/project_4')
if my_path.is_dir():
    df_0 = pd.read_csv('/home/klarazetkin/Documents/yandex/module_2/project_4/geo_data_0.csv')
    df_1 = pd.read_csv('/home/klarazetkin/Documents/yandex/module_2/project_4/geo_data_1.csv')
    df_2 = pd.read_csv('/home/klarazetkin/Documents/yandex/module_2/project_4/geo_data_2.csv')
else:
    df_0 = pd.read_csv('/datasets/geo_data_0.csv')
    df_1 = pd.read_csv('/datasets/geo_data_1.csv')
    df_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [157]:
display(df_0.head())
display(df_1.head())
display(df_2.head())

,id,f0,f1,f2,product
0,txEyH,0.7057,-0.4978,1.2212,105.2801
1,2acmU,1.3347,-0.3402,4.3651,73.0378
2,409Wp,1.0227,0.1520,1.4199,85.2656
3,iJLyR,-0.0322,0.1390,2.9786,168.6208
4,Xdl7t,1.9884,0.1554,4.7518,154.0366


,id,f0,f1,f2,product
0,kBEdx,-15.0013,-8.2760,-0.0059,3.1791
1,62mP7,14.2721,-3.4751,0.9992,26.9533
2,vyE1P,6.2632,-5.9484,5.0012,134.7663
3,KcrkZ,-13.0812,-11.5061,4.9994,137.9454
4,AHL4O,12.7022,-8.1474,5.0044,134.7663


,id,f0,f1,f2,product
0,fwXo0,-1.1470,0.9633,-0.8290,27.7587
1,WJtFt,0.2628,0.2698,-2.5302,56.0697
2,ovLUW,0.1946,0.2890,-5.5864,62.8719
3,q6cA6,2.2361,-0.5538,0.9300,114.5728
4,WPMUX,-0.5160,1.7163,5.8990,149.6007


In [158]:
print(df_0.shape)
print(df_1.shape)
print(df_2.shape)

(100000, 5)
(100000, 5)
(100000, 5)


In [159]:
print(df_0.info())
print(df_1.info())
print(df_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory 

In [160]:
print(df_0.isna().mean())
print(df_1.isna().mean())
print(df_2.isna().mean())

id        0.0000
f0        0.0000
f1        0.0000
f2        0.0000
product   0.0000
dtype: float64
id        0.0000
f0        0.0000
f1        0.0000
f2        0.0000
product   0.0000
dtype: float64
id        0.0000
f0        0.0000
f1        0.0000
f2        0.0000
product   0.0000
dtype: float64


# Подготовка данных
## Удаление неинформативных столбцов
Столбцы с `id` неинформативны, обучать на них модель не нужно. Удалим их.

In [161]:
df_0 = df_0.drop('id', axis=1)
df_1 = df_1.drop('id', axis=1)
df_2 = df_2.drop('id', axis=1)

## Выделение обучающей и валидационной выборки
Разделим датасет на обучающую и валидационную выборки в соотношении 75 : 25. Укажем параметр `random_state=666` для обеспечения повторяемости результата.

In [162]:
df_0_train, df_0_valid = train_test_split(df_0, test_size=0.25, random_state=666)
df_1_train, df_1_valid = train_test_split(df_1, test_size=0.25, random_state=666)
df_2_train, df_2_valid = train_test_split(df_2, test_size=0.25, random_state=666)

## Определение целевого признака и других признаков
Целевой признак - объем запасов в скважине (тыс. баррелей), он находится в колонке `product`. Выделим целевой и обучающие признаки для каждого датасета.

In [163]:
df_0_features_train = df_0_train.drop('product', axis=1)
df_0_target_train = df_0_train['product']
df_0_features_valid = df_0_valid.drop('product', axis=1)
df_0_target_valid = df_0_valid['product']

df_1_features_train = df_1_train.drop('product', axis=1)
df_1_target_train = df_1_train['product']
df_1_features_valid = df_1_valid.drop('product', axis=1)
df_1_target_valid = df_1_valid['product']

df_2_features_train = df_2_train.drop('product', axis=1)
df_2_target_train = df_2_train['product']
df_2_features_valid = df_2_valid.drop('product', axis=1)
df_2_target_valid = df_2_valid['product']

# Создание и обучение моделей
Создадим и обучим модели линейной регрессии.

RMSE нулевой и второй модели составляет 40.7% и 41.98% от среднего реального объема сырья. Это очень много. Однако линейная регрессия справилась немного лучше, чем "модель", которая всегда предсказывает среднее, продолжаем работать с ней.

Намного удачней получилась модель по df_1, ее RMSE составил 0.89 против 46.11 у среднего. Но в первом регионе меньше средний реальный запас сырья.

In [164]:
def get_dummy_predictions(target_train, features_valid):
    mean_value = target_train.mean()
    predictions = pd.Series([mean_value] * len(features_valid))
    return predictions

In [165]:
def teach_model(features_train, target_train, features_valid, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = pd.Series(model.predict(features_valid))
    
    mean_predicted_product = predictions_valid.mean()
    print("Средний предсказанный запас сырья:", mean_predicted_product)
    
    mean_real_product = target_valid.mean()
    print("Средний реальный запас сырья:", mean_real_product)
    
    this_rmse = mean_squared_error(target_valid, predictions_valid) ** 0.5 
    print("RMSE модели линейной регрессии на валидационной выборке:", this_rmse)
    
    dummy_predictions = get_dummy_predictions(target_train, features_valid)
    rmse_of_dummy_predictions = mean_squared_error(target_valid, dummy_predictions) ** 0.5
    print("RMSE модели, предсказывающей среднее:", rmse_of_dummy_predictions)
    print()
    return model, predictions_valid


In [166]:
model_0, df_0_predictions_valid = teach_model(
    df_0_features_train, df_0_target_train, df_0_features_valid, df_0_target_valid)

model_1, df_1_predictions_valid = teach_model(
    df_1_features_train, df_1_target_train, df_1_features_valid, df_1_target_valid)

model_2, df_2_predictions_valid = teach_model(
    df_2_features_train, df_2_target_train, df_2_features_valid, df_2_target_valid)

Средний предсказанный запас сырья: 92.59883747361133
Средний реальный запас сырья: 92.7335845019874
RMSE модели линейной регрессии на валидационной выборке: 37.78852914975086
RMSE модели, предсказывающей среднее: 44.34373355040947

Средний предсказанный запас сырья: 69.119604542144
Средний реальный запас сырья: 69.12651161786542
RMSE модели линейной регрессии на валидационной выборке: 0.8899276270922475
RMSE модели, предсказывающей среднее: 46.119134855081626

Средний предсказанный запас сырья: 95.05831290258574
Средний реальный запас сырья: 95.29994745917732
RMSE модели линейной регрессии на валидационной выборке: 40.008583723061925
RMSE модели, предсказывающей среднее: 44.54993257842992



## Расчет прибыли, необходимой для безубыточной разработки
Рассчитаем объем запасов сырья, необходмый для безубыточной разработки. 

Бюджет на разработку скважин в регионе — 10 млрд. рублей. При нынешних ценах один баррель сырья приносит 450 рублей дохода. В датафреймах доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей. Значения для расчётов сохраним в отдельных переменных.

Если считать, что расходы на разработку всех скважин одинаковые, то средний достаточный объём сырья для безубыточной разработки новой скважины составляет 111.11 тысяч баррелей. Это значительно больше, чем доступный средний реальный запас сырья. На самом деле так, скорей всего, не бывает и бюджет определен для всего региона.

Чтоб разработка не была убыточной, нужно, чтоб общий объем запасов сырья по 200 крупнейшим скважинам региона был не менее 22222.22 тысяч баррелей. Если хотя бы приблизительно верно выбрать скважины, то разработка окупится в любом регионе.

> Объем запасов сырья по нулевому региону (топ 200): 36966.747929072015

> Объем запасов сырья по первому региону (топ 200): 27589.081548181137

> Объем запасов сырья по второму региону (топ 200): 37910.29539635329

In [167]:
TOTAL_BUDGET = 10_000_000 # тысяч рублей
PRICE = 450 # тысяч рублей за тысячу баррелей
WELLS_NUMBER = 200 # всего нужно разработать 200 скважин
WELL_BUDGET = TOTAL_BUDGET / WELLS_NUMBER
ONE_WELL_DEMANDED_VOLUME = WELL_BUDGET / PRICE
TOTAL_DEMANDED_VOLUME = TOTAL_BUDGET / PRICE 

print("Необходимый объем с одной скважины:", ONE_WELL_DEMANDED_VOLUME)
print("Необходимый объем с региона:", TOTAL_DEMANDED_VOLUME)

Необходимый объем с одной скважины: 111.11111111111111
Необходимый объем с региона: 22222.222222222223


In [168]:
# передаем любой датафрейм, считаем объем сырья по 200 наиболее выгодным точкам
def biggest_200_product(df):
    biggest_200_product = df['product'].sort_values(ascending=False)[:WELLS_NUMBER].sum()
    return biggest_200_product

print("Объем запасов сырья по нулевому региону (топ 200):", biggest_200_product(df_0))
print("Объем запасов сырья по первому региону (топ 200):", biggest_200_product(df_1))
print("Объем запасов сырья по второму региону (топ 200):", biggest_200_product(df_2))


Объем запасов сырья по нулевому региону (топ 200): 36966.747929072015
Объем запасов сырья по первому региону (топ 200): 27589.081548181137
Объем запасов сырья по второму региону (топ 200): 37910.29539635329


In [169]:
def random_200_product(df):
    random_200_product = df.sample(n=200, replace=False, random_state=666)['product'].sum()
    return random_200_product
    
print("Объем запасов сырья по нулевому региону (топ 200):", random_200_product(df_0))
print("Объем запасов сырья по первому региону (топ 200):", random_200_product(df_1))
print("Объем запасов сырья по второму региону (топ 200):", random_200_product(df_2))

Объем запасов сырья по нулевому региону (топ 200): 19180.902377384202
Объем запасов сырья по первому региону (топ 200): 12735.484948328225
Объем запасов сырья по второму региону (топ 200): 18930.103303645927


## Расчёт прибыли по выбранным скважинам: факт и предсказания модели
### Переиндексация
Для удобства работы заменим индексы в объектах Series c предсказанным целевым признаком.

In [170]:
# actually, I pass Series to the function
def set_new_indices(df1, df2):
    new = pd.DataFrame(df1)
    new['new_index'] = list(df2.index)
    new = new.set_index('new_index')
    new.columns = ['product']
    return new

In [171]:
df_0_predictions_valid = set_new_indices(df_0_predictions_valid, df_0_target_valid)
df_1_predictions_valid = set_new_indices(df_1_predictions_valid, df_1_target_valid)
df_2_predictions_valid = set_new_indices(df_2_predictions_valid, df_2_target_valid)

### Функция расчета прибыли по лучшим скважинам
Напишем функцию расчета прибыли по лучшим скважинам. На вход она принимает целевой признак и соответствующие ему предсказания модели. 

Если выбрать скважины, по которым предсказан максимальный объем сырья, то разница между предсказанным и реальным суммарным объемом сырья по 200 лучшим скважинам составляет от 0.6 до 10.5 %. Причем модель завышает цифры, фактические значения меньше.

In [172]:
def get_target_revenue(target, predictions):
    top_200_predictions = predictions.sort_values(by='product', ascending=False)[:WELLS_NUMBER] 
    target_product_for_chosen_wells = target.loc[top_200_predictions.index].sum()
    predicted_product_for_chosen_wells = top_200_predictions['product'].sum()
    difference = (
        (target_product_for_chosen_wells - predicted_product_for_chosen_wells) / 
        predicted_product_for_chosen_wells * 100)
    print("Предсказанный объем:", predicted_product_for_chosen_wells)
    print("Реальный объем:", target_product_for_chosen_wells)
    print("Разница в % :", difference)
    target_revenue = target_product_for_chosen_wells * PRICE
    print('Реальная выручка:', target_revenue)
    return target_revenue

In [173]:
print('df_0:') 
get_target_revenue(df_0_target_valid, df_0_predictions_valid)

print('df_1:') 
get_target_revenue(df_1_target_valid, df_1_predictions_valid)

print('df_2:') 
get_target_revenue(df_2_target_valid, df_2_predictions_valid)


df_0:
Предсказанный объем: 30958.987116501394
Реальный объем: 29700.236213571203
Разница в % : -4.065865908963366
Реальная выручка: 13365106.29610704
df_1:
Предсказанный объем: 27744.859968916986
Реальный объем: 27589.081548181137
Разница в % : -0.5614676769332054
Реальная выручка: 12415086.696681513
df_2:
Предсказанный объем: 29781.910929229674
Реальный объем: 26668.42115931984
Разница в % : -10.454298172163545
Реальная выручка: 12000789.521693928


12000789.521693928

## Расчет рисков и прибыли для каждого региона методом Bootstrap
### Функция для расчета прибыли техникой Bootstrap
При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки. Это означает, что при промышленном использовании модели могут попасться 500 любых точек из известного региона, более выгодных или менее выгодных.

Чтоб рассчитать возможные риски и прибыль для каждого региона, применим технику Bootstrap:

создадим функцию `get_randomized_profits()`, а в ней создадим 1000 случайных выборок из 500 месторождений каждая; из 500 точек выберем 200 лучших и посчитаем реальную выручку от их разработки. 

Для функции `sample` укажем параметр `replace=True`, чтоб элементы попадали в выборку "с возвращением", т.е. совершенно случайно (если указать `replace=False`, то вероятности попадания элементов в выборку будет изменяться с каждым циклом).

### Результаты

Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков. Полученные данные такие:

**Регион 0:**
* Средняя прибыль: 432463.98931010463
* Доверительный интервал 95% - от -79795.73523867916 до 933215.6988282925
* Риск убытка: 4.1

**Регион 1:**
* Средняя прибыль: 485311.99327859236
* Доверительный интервал 95% - от 78851.63360354537 до 871588.5070965604
* Риск убытка: 1.2

**Регион 2:**
* Средняя прибыль: 356673.2211995138
* Доверительный интервал 95% - от -163533.79410967336 до 859354.0850115806
* Риск убытка: 7.8


По условию задачи, после оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2,5 %, и среди них выбрать регион с наибольшей средней прибылью. 

Единственный регион, где риск убытков меньше 2,5 %, - это первый регион, риск убытков в нем составляет 1.2 %. Средняя прибыль по 200 крупнейшим скважинам из случайных 500 в нем тоже максимальная и составляет 485_311.99 тысяч рублей. В то же время средний объем сырья по всем скважинам региона (а значит и средняя прибыль по всем скважинам региона) в нем минимален и составляет 69.13 тысяч баррелей на одну скважину.

In [174]:
def get_top_200_target_profit(target, predictions):
    top_200_predictions = predictions.sort_values(by='product', ascending=False)[:WELLS_NUMBER] 
    target_product_for_chosen_wells = target.loc[top_200_predictions.index].sum()
    target_profit = target_product_for_chosen_wells * PRICE - TOTAL_BUDGET
    return target_profit

In [175]:
# provide full predictions and full target for a region
state = RandomState(666)
def get_randomized_profits(target, predictions):
    target_profits = [] # here we keep summed target profits for predicted top 200 wells
    for i in range(1000):
        predictions_subsample = predictions.sample(n=500, replace=True, random_state=state)
        target_profit = get_top_200_target_profit(target, predictions_subsample)
        target_profits.append(target_profit)
    
    return pd.DataFrame(target_profits)
        
    

In [176]:
df_0_random_real_profits = get_randomized_profits(df_0_target_valid, df_0_predictions_valid)
df_1_random_real_profits = get_randomized_profits(df_1_target_valid, df_1_predictions_valid)
df_2_random_real_profits = get_randomized_profits(df_2_target_valid, df_2_predictions_valid)

In [181]:
def count_final_values(profits):
    mean_profit = profits.mean()[0]
    lower_0025 = float(profits.quantile(0.025))
    upper_0025 = float(profits.quantile(0.975))
    loss_probability = float(profits[profits[0] < 0].count() / len(profits))
    print("Средняя прибыль:", mean_profit)
    print("Доверительный интервал 95% - от", lower_0025, "до", upper_0025)
    print("Риск убытка:", loss_probability * 100)
    return mean_profit, loss_probability, lower_0025, upper_0025

In [182]:
print("Регион 0:")
df_0_mean_profit, df_0_loss_probability, df_0_lower_0025, df_0_upper_0025 = \
    count_final_values(df_0_random_real_profits)
print()

print("Регион 1:")
df_1_mean_profit, df_1_loss_probability, df_1_lower_0025, df_1_upper_0025 = \
    count_final_values(df_1_random_real_profits)
print()

print("Регион 2:")
df_2_mean_profit, df_2_loss_probability, df_2_lower_0025, df_2_upper_0025 = \
    count_final_values(df_2_random_real_profits)

Регион 0:
Средняя прибыль: 432463.98931010463
Доверительный интервал 95% - от -79795.73523867916 до 933215.6988282925
Риск убытка: 4.1000000000000005

Регион 1:
Средняя прибыль: 485311.99327859236
Доверительный интервал 95% - от 78851.63360354537 до 871588.5070965604
Риск убытка: 1.2

Регион 2:
Средняя прибыль: 357729.9431703274
Доверительный интервал 95% - от -148545.54286148874 до 860627.8598465608
Риск убытка: 7.8


In [183]:
output_table = pd.DataFrame(
    [["Нулевой регион", 
      df_0_mean_profit, 
      df_0_loss_probability * 100, 
      "от  " + str(round(df_0_lower_0025,2)) + "  до  " + str(round(df_0_upper_0025, 2))],
     
     ["Первый регион", 
      df_1_mean_profit, 
      df_1_loss_probability * 100, 
      "от  " + str(round(df_1_lower_0025,2)) + "  до  " + str(round(df_1_upper_0025, 2))],
     
     ["Второй регион", 
      df_2_mean_profit, 
      df_2_loss_probability * 100, 
     "от  " + str(round(df_2_lower_0025,2)) + "  до  " + str(round(df_2_upper_0025, 2))]
    ],columns=["Регион", "Средняя прибыль", "Риск убытка, %", "Доверительный интервал 95 %"]
)
display(output_table)


,Регион,Средняя прибыль,"Риск убытка, %",Доверительный интервал 95 %
0,Нулевой регион,"432,463.9893",4.1000,от -79795.74 до 933215.7
1,Первый регион,"485,311.9933",1.2000,от 78851.63 до 871588.51
2,Второй регион,"357,729.9432",7.8000,от -148545.54 до 860627.86


## Вывод
Для решения задачи выбора региона для разработки нефти на датасетах с данными о месторождениях были обучены модели линейной регрессии. 

Методом Bootstrap была смоделирована реальная ситуация, когда при разведке региона исследуется 500 точек, из которых с помощью обученной модели выбирается 200 лучших для разработки. Таким образом, была оценена предполагаемая прибыль и риск убытков при выборе 200 лучших точек из 500 случайных точек, принадлежащих региону.

Полученные результаты представлены в таблице:

In [184]:
output_table

,Регион,Средняя прибыль,"Риск убытка, %",Доверительный интервал 95 %
0,Нулевой регион,"432,463.9893",4.1000,от -79795.74 до 933215.7
1,Первый регион,"485,311.9933",1.2000,от 78851.63 до 871588.51
2,Второй регион,"357,729.9432",7.8000,от -148545.54 до 860627.86


По условию задачи, после оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2,5 %, и среди них выбрать регион с наибольшей средней прибылью. 

Единственный регион, где риск убытков меньше 2,5 %, - это первый регион, риск убытков в нем составляет 1,2 %. Средняя прибыль по 200 крупнейшим скважинам из случайных 500 в нем тоже максимальная и составляет 485_311.99 тысяч рублей. 

Модель, обученная по данным первого региона, наиболее точная из всех трех, она показывает наименьшую RMSE на валидационной выборке: 

> RMSE модели линейной регрессии на валидационной выборке: 0.8899276270922485

> RMSE модели, предсказывающей среднее: 46.119134855081626

Это значит, что предсказания по этому региону наиболее точны, что снижает риск убытков.

В то же время средний объем сырья по всем скважинам региона (а значит и средняя прибыль по всем скважинам региона) в нем минимален и составляет 69.13 тысяч баррелей на скважину. Минимален также и общий запас сырья крупнейших 200 скважин региона - он составляет 27_589.08 тысяч баррелей.

К разработке рекомендован первый регион как единственный подходящий под формулировку задачи. 